<a href="https://colab.research.google.com/github/GuyInFreezer/project-2/blob/Yeong-branch/Datafram_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Dependencies
import requests
import json
import pandas as pd
from google.colab import userdata
import time

# Import API key from secret data
api_key = userdata.get('POLYGONKEY')

# Setup basics for the polygon API
base_url = 'https://api.polygon.io'
stock_names = ['SPY', 'QQQ', 'VXX', 'DIA']
start_date = '2022-01-03'
end_date = '2024-03-22'
start_date_5d = '2021-12-27'

In [ ]:
# ###DF structure###

# Timestamp

# vvvvvvvvvvvvvvvv repeat for each stock vvvvvvvvvvvvvvvvvvv
# 5D Change [Open 5 days ago - Close 1 day ago] / Open 5 days ago
# 3D change [Open 3 days ago - Close 1 day ago] / Open 3 days ago
# 1D change [Open 1 days ago - Close 1 day ago] / Open 1 days ago
# Stock Price at 9:35 (Lowest Price)
# Stock Price at 15:45 (Lowest Price)
# Strike - Ceil above
# Covered Call at 9:35 (Lowest Price)
# Covered Call at 15:45 (Highest Price)
# 5D Avg Total Volume
# 3D Avg Total Volume
# 1D Avg Total Volume
# ^^^^^^^^^^^^^^^ repeat for each stock ^^^^^^^^^^^^^^^^^^^^
#
# Net - SPY Closing Stock Price + (SPY CC sold[open] - SPY CC buyback[close])
#	   if (SPY Closing stock price - SPY Opening stock price) is negative, SPY Closing Stock Price + (SPY CC sold[open])
#	   y = net / SPY Stock Price at 9:35
#
#
# Need 5 DFs
#
# 5 Days before at 1D interval
# 5 Days before at 5M interval
# Regular at 1D interval
# Regular at 5M interval
# Regular Options at 5M interval

In [ ]:
# Step 1 - Grab Regular Stock Data at 1D Interval

# Create empty DF to merge
df_stock_1d = pd.DataFrame()

for stocksTicker in stock_names:
  # Generate Quert URL
  query_url = f"{base_url}/v2/aggs/ticker/{stocksTicker}/range/1/day/{start_date}/{end_date}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}"

  # Grab JSON
  response = requests.get(query_url)
  json_data = response.json()

  # Only append DF if the result isn't empty
  if json_data['resultsCount'] > 0:
    # Convert JSON to Pandas Dataframe
    temp_df = pd.json_normalize(json_data['results'])
    # Rename columns
    temp_df = temp_df.rename(columns={'v':'Volume', 'vw':'Volume Weighted', 'o':'Open Price', 'c':'Close Price', 'h':'Highest Price', 'l':'Lowest Price', 't':'Timestamp', 'n':'Number of Transaction'})
    # Convert microsecond timestamp to Pandas Timestamp. We're only interested in Date so only grab date
    temp_df['Timestamp'] = pd.to_datetime(temp_df['Timestamp'], unit='ms').dt.date # We won't need .dt.date for 5 minute interval one because we actually want the time.
    # Add stock name column for visibility
    temp_df['Stock Name'] = stocksTicker

    # Concat the temp_df to the main DF
    df_stock_1d = pd.concat([df_stock_1d, temp_df], axis = 0, ignore_index = True)

# Review DF
df_stock_1d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2232 entries, 0 to 2231
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Volume                 2232 non-null   float64
 1   Volume Weighted        2232 non-null   float64
 2   Open Price             2232 non-null   float64
 3   Close Price            2232 non-null   float64
 4   Highest Price          2232 non-null   float64
 5   Lowest Price           2232 non-null   float64
 6   Timestamp              2232 non-null   object 
 7   Number of Transaction  2232 non-null   int64  
 8   Stock Name             2232 non-null   object 
dtypes: float64(6), int64(1), object(2)
memory usage: 157.1+ KB


In [ ]:
# Step 2 - Grab Regular Stock Data at 5M Interval (To be done by Mat)

# To Mat - Do the same thing as above, but change '/range/1/day' in query_url to '/range/5/minute'.
# Also, change the DF name from df_stock_1d to df_stock_5m

### Start code from here ###
# Create empty DF to merge
df_stock_5m_5d = pd.DataFrame()

timedelta = 814

for td in range (0, timedelta + 1):
  for stocksTicker in stock_names:
    sd = (pd.to_datetime(start_date_5d) + pd.Timedelta(td, 'd')).strftime('%Y-%m-%d')
    # Generate Query URL
    query_url = f"{base_url}/v2/aggs/ticker/{stocksTicker}/range/5/minute/{sd}/{sd}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}"

    # Grab JSON
    response = requests.get(query_url)
    json_data = response.json()
    # Only append DF if the result isn't empty
    if json_data['resultsCount'] > 0:
      # Convert JSON to Pandas Dataframe
      temp_df = pd.json_normalize(json_data['results'])
      # Rename columns
      temp_df = temp_df.rename(columns={'v':'Volume', 'vw':'Volume Weighted', 'o':'Open Price', 'c':'Close Price', 'h':'Highest Price', 'l':'Lowest Price', 't':'Timestamp', 'n':'Number of Transaction'})
      # Convert microsecond timestamp to Pandas Timestamp. We're only interested in Date so only grab date
      temp_df['Timestamp'] = pd.to_datetime(temp_df['Timestamp'], unit='ms') # We won't need .dt.date for 5 minute interval one because we actually want the time.
      # Add stock name column for visibility
      temp_df['Stock Name'] = stocksTicker

      # Concat the temp_df to the main DF
      df_stock_5m_5d = pd.concat([df_stock_5m_5d, temp_df], axis = 0, ignore_index = True)

# Review DF
df_stock_5m_5d.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359385 entries, 0 to 359384
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Volume                 359385 non-null  float64       
 1   Volume Weighted        359385 non-null  float64       
 2   Open Price             359385 non-null  float64       
 3   Close Price            359385 non-null  float64       
 4   Highest Price          359385 non-null  float64       
 5   Lowest Price           359385 non-null  float64       
 6   Timestamp              359385 non-null  datetime64[ns]
 7   Number of Transaction  359385 non-null  int64         
 8   Stock Name             359385 non-null  object        
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 24.7+ MB


In [ ]:
# Step 3 - Grab 5-Days before Stock Data at 1D Interval

# Create empty DF to merge
df_stock_1d_5d = pd.DataFrame()

for stocksTicker in stock_names:
  # Generate Quert URL
  query_url = f"{base_url}/v2/aggs/ticker/{stocksTicker}/range/1/day/{start_date_5d}/{end_date}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}"

  # Grab JSON
  response = requests.get(query_url)
  json_data = response.json()

  # Only append DF if the result isn't empty
  if json_data['resultsCount'] > 0:
    # Convert JSON to Pandas Dataframe
    temp_df = pd.json_normalize(json_data['results'])
    # Rename columns
    temp_df = temp_df.rename(columns={'v':'Volume', 'vw':'Volume Weighted', 'o':'Open Price', 'c':'Close Price', 'h':'Highest Price', 'l':'Lowest Price', 't':'Timestamp', 'n':'Number of Transaction'})
    # Convert microsecond timestamp to Pandas Timestamp. We're only interested in Date so only grab date
    temp_df['Timestamp'] = pd.to_datetime(temp_df['Timestamp'], unit='ms').dt.date # We won't need .dt.date for 5 minute interval one because we actually want the time.
    # Add stock name column for visibility
    temp_df['Stock Name'] = stocksTicker

    # Concat the temp_df to the main DF
    df_stock_1d_5d = pd.concat([df_stock_1d_5d, temp_df], axis = 0, ignore_index = True)

# Review DF
df_stock_1d_5d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2252 entries, 0 to 2251
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Volume                 2252 non-null   float64
 1   Volume Weighted        2252 non-null   float64
 2   Open Price             2252 non-null   float64
 3   Close Price            2252 non-null   float64
 4   Highest Price          2252 non-null   float64
 5   Lowest Price           2252 non-null   float64
 6   Timestamp              2252 non-null   object 
 7   Number of Transaction  2252 non-null   int64  
 8   Stock Name             2252 non-null   object 
dtypes: float64(6), int64(1), object(2)
memory usage: 158.5+ KB


In [ ]:
# Step 4 - Grab 5-Days before Stock Data a 5M Interval (To be done by Mat)

df_stock_5m = df_stock_5m_5d.loc[df_stock_5m_5d['Timestamp'].dt.date >= pd.to_datetime(start_date)]

# Review DF
df_stock_5m.info()


<ipython-input-5-92bd1a81c586>:3: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df_stock_5m = df_stock_5m_5d.loc[df_stock_5m_5d['Timestamp'].dt.date >= pd.to_datetime(start_date)]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 356159 entries, 3226 to 359384
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Volume                 356159 non-null  float64       
 1   Volume Weighted        356159 non-null  float64       
 2   Open Price             356159 non-null  float64       
 3   Close Price            356159 non-null  float64       
 4   Highest Price          356159 non-null  float64       
 5   Lowest Price           356159 non-null  float64       
 6   Timestamp              356159 non-null  datetime64[ns]
 7   Number of Transaction  356159 non-null  int64         
 8   Stock Name             356159 non-null  object        
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 27.2+ MB


In [ ]:
# Step 4.5 - Strain 5M Interval DF so that only 13:35:00 UTC (9:35 AM EST) and 19:45:00 UTC (3:45 PM EST) are left.
# We don't need any other timeframe.

df_stock_open_5d = df_stock_5m_5d.loc[(df_stock_5m_5d['Timestamp'].dt.hour == 13) & (df_stock_5m_5d['Timestamp'].dt.minute == 35)].reset_index()
df_stock_open_5d = df_stock_open_5d[['Lowest Price', 'Timestamp', 'Stock Name']].reset_index()
df_stock_open_5d['Timestamp'] = df_stock_open_5d['Timestamp'].dt.date
df_stock_open = df_stock_open_5d.loc[pd.to_datetime(df_stock_open_5d['Timestamp']) >= pd.to_datetime(start_date)]

df_stock_close_5d = df_stock_5m_5d.loc[(df_stock_5m_5d['Timestamp'].dt.hour == 19) & (df_stock_5m_5d['Timestamp'].dt.minute == 45)].reset_index()
df_stock_close_5d = df_stock_close_5d[['Lowest Price', 'Timestamp', 'Stock Name']].reset_index()
df_stock_close_5d['Timestamp'] = df_stock_close_5d['Timestamp'].dt.date
df_stock_close = df_stock_close_5d.loc[pd.to_datetime(df_stock_close_5d['Timestamp']) >= pd.to_datetime(start_date)]

df_stock_close_5d.loc[df_stock_close_5d['Stock Name'] == 'SPY'].head(20)


,index,Lowest Price,Timestamp,Stock Name
0,0,476.0000,2021-12-27,SPY
4,4,476.4800,2021-12-28,SPY
8,8,477.3000,2021-12-29,SPY
12,12,478.0900,2021-12-30,SPY
16,16,476.0100,2021-12-31,SPY
20,20,477.3800,2022-01-03,SPY
24,24,477.1700,2022-01-04,SPY
28,28,471.1100,2022-01-05,SPY
32,32,468.9900,2022-01-06,SPY
36,36,467.1100,2022-01-07,SPY


In [ ]:
df_stock_1d_5d['Volume'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2252 entries, 0 to 2251
Series name: Volume
Non-Null Count  Dtype  
--------------  -----  
2252 non-null   float64
dtypes: float64(1)
memory usage: 17.7 KB


In [ ]:
# Step 5 - Begin creating base DF

from math import isnan

### Columns to be worked on this step ###

# Timestamp [X]
# 5D Avg Total Volume [X]
# 3D Avg Total Volume [WIP] << Weird with weekends data being non-existant
# 1D Avg Total Volume [WIP] << Weird with weekends data being non-existant
# 5D Change [Open 5 days ago - Close 1 day ago] / Open 5 days ago [X]
# 3D change [Open 3 days ago - Close 1 day ago] / Open 3 days ago [WIP] << Weird with weekends data being non-existant
# 1D change [Open 1 days ago - Close 1 day ago] / Open 1 days ago [WIP] << Weird with weekends data being non-existant
# Stock Price at 9:35 (Lowest Price) [X]
# Stock Price at 15:45 (Lowest Price) [X]
# Strike - Ceil above [X]

### Timestamp
df = df_stock_1d.copy()
df = df.loc[df['Stock Name'] == 'SPY']
df = df[['Timestamp']]

### Stock Price at 9:35
# Grab date + time
#
# Do .loc on df_stock_5m to grab ['Lowest Price']
#
# Profit
temp = df_stock_open.loc[df_stock_open['Stock Name'] == 'SPY']
temp = temp.drop(labels=['Stock Name', 'index'], axis = 1)
df = pd.merge(df, temp, on=['Timestamp'])
df = df.rename(columns={'Lowest Price':'SPY Opening Price'})

temp = df_stock_open.loc[df_stock_open['Stock Name'] == 'QQQ']
temp = temp.drop(labels=['Stock Name', 'index'], axis = 1)
df = pd.merge(df, temp, on=['Timestamp'])
df = df.rename(columns={'Lowest Price':'QQQ Opening Price'})

temp = df_stock_open.loc[df_stock_open['Stock Name'] == 'VXX']
temp = temp.drop(labels=['Stock Name', 'index'], axis = 1)
df = pd.merge(df, temp, on=['Timestamp'])
df = df.rename(columns={'Lowest Price':'VXX Opening Price'})

temp = df_stock_open.loc[df_stock_open['Stock Name'] == 'DIA']
temp = temp.drop(labels=['Stock Name', 'index'], axis = 1)
df = pd.merge(df, temp, on=['Timestamp'])
df = df.rename(columns={'Lowest Price':'DIA Opening Price'})


### Stock Price at 15:45
# Grab date + time
#
# Do .loc on df_stock_5m to grab ['Lowest Price']
#
# Profit
temp = df_stock_close.loc[df_stock_close['Stock Name'] == 'SPY']
temp = temp.drop(labels=['Stock Name', 'index'], axis = 1)
df = pd.merge(df, temp, on=['Timestamp'])
df = df.rename(columns={'Lowest Price':'SPY Closing Price'})

temp = df_stock_close.loc[df_stock_close['Stock Name'] == 'QQQ']
temp = temp.drop(labels=['Stock Name', 'index'], axis = 1)
df = pd.merge(df, temp, on=['Timestamp'])
df = df.rename(columns={'Lowest Price':'QQQ Closing Price'})

temp = df_stock_close.loc[df_stock_close['Stock Name'] == 'VXX']
temp = temp.drop(labels=['Stock Name', 'index'], axis = 1)
df = pd.merge(df, temp, on=['Timestamp'])
df = df.rename(columns={'Lowest Price':'VXX Closing Price'})

temp = df_stock_close.loc[df_stock_close['Stock Name'] == 'DIA']
temp = temp.drop(labels=['Stock Name', 'index'], axis = 1)
df = pd.merge(df, temp, on=['Timestamp'])
df = df.rename(columns={'Lowest Price':'DIA Closing Price'})


### Avg Volumes
# Function to be applied for changes (Using df_stock_1d_5d for data)

temp = df_stock_1d_5d.loc[df_stock_1d_5d['Stock Name'] == 'SPY']

def avg_volume_5d (row):
  if (pd.to_datetime(row['Timestamp']) - pd.Timedelta(1, 'd')).dayofweek < 5:
    result = temp.loc[ (pd.to_datetime(temp['Timestamp']) >= (pd.to_datetime(row['Timestamp']) - pd.Timedelta(7, 'd'))) &\
                              (pd.to_datetime(temp['Timestamp']) <= (pd.to_datetime(row['Timestamp']) - pd.Timedelta(1, 'd'))) ]['Volume'].mean()
  else:
    # Weekend check
    result = temp.loc[ (pd.to_datetime(temp['Timestamp']) >= (pd.to_datetime(row['Timestamp']) - pd.Timedelta(7, 'd'))) &\
                              (pd.to_datetime(temp['Timestamp']) <= (pd.to_datetime(row['Timestamp']) - pd.Timedelta(3, 'd'))) ]['Volume'].mean()
  return result
#
def avg_volume_3d (row):
  date_a = 1
  date_b = 3
  dow_a = (pd.to_datetime(row['Timestamp']) - pd.Timedelta(1, 'd')).dayofweek
  dow_b = (pd.to_datetime(row['Timestamp']) - pd.Timedelta(3, 'd')).dayofweek

  if (dow_a < 5) & (dow_b < 5): # Neither are weekends
    date_a = 1
    date_b = 3
  elif (dow_a >= 5): # Day before is Sunday, so 3-days before cannot be weekend
    date_a = 3
    date_b = 5
  else: # 3-days before is weekend
    date_a = 1
    date_b = 3 + dow_b - 4

  result = temp.loc[ (pd.to_datetime(temp['Timestamp']) >= (pd.to_datetime(row['Timestamp']) - pd.Timedelta(date_b, 'd'))) &\
                              (pd.to_datetime(temp['Timestamp']) <= (pd.to_datetime(row['Timestamp']) - pd.Timedelta(date_a, 'd'))) ]['Volume'].mean()
  return result

def avg_volume_1d (row):
  if (pd.to_datetime(row['Timestamp']) - pd.Timedelta(1, 'd')).dayofweek < 5:
      result = temp.loc[ (pd.to_datetime(temp['Timestamp']) <= (pd.to_datetime(row['Timestamp']) - pd.Timedelta(1, 'd'))) ]['Volume'].mean()
  else:
      # Weekend check
      result = temp.loc[ (pd.to_datetime(temp['Timestamp']) <= (pd.to_datetime(row['Timestamp']) - pd.Timedelta(3, 'd'))) ]['Volume'].mean()
  return result
#
#
# Uncomment below when Avg volume functions are written out
df['SPY 5D Avg Volume'] = df.apply(avg_volume_5d, axis = 1)
df['SPY 3D Avg Volume'] = df.apply(avg_volume_3d, axis = 1)
df['SPY 1D Avg Volume'] = df.apply(avg_volume_1d, axis = 1)
temp = df_stock_1d_5d.loc[df_stock_1d_5d['Stock Name'] == 'QQQ']
df['QQQ 5D Avg Volume'] = df.apply(avg_volume_5d, axis = 1)
df['QQQ 3D Avg Volume'] = df.apply(avg_volume_3d, axis = 1)
df['QQQ 1D Avg Volume'] = df.apply(avg_volume_1d, axis = 1)
temp = df_stock_1d_5d.loc[df_stock_1d_5d['Stock Name'] == 'VXX']
df['VXX 5D Avg Volume'] = df.apply(avg_volume_5d, axis = 1)
df['VXX 3D Avg Volume'] = df.apply(avg_volume_3d, axis = 1)
df['VXX 1D Avg Volume'] = df.apply(avg_volume_1d, axis = 1)
temp = df_stock_1d_5d.loc[df_stock_1d_5d['Stock Name'] == 'DIA']
df['DIA 5D Avg Volume'] = df.apply(avg_volume_5d, axis = 1)
df['DIA 3D Avg Volume'] = df.apply(avg_volume_3d, axis = 1)
df['DIA 1D Avg Volume'] = df.apply(avg_volume_1d, axis = 1)

### Stock Price Change %

temp = df_stock_open_5d.loc[df_stock_open_5d['Stock Name'] == 'SPY']
temp2 = df_stock_close_5d.loc[df_stock_close_5d['Stock Name'] == 'SPY']
def stock_change_5d (row):
  date_a = 1
  date_b = 7
  dow_a = (pd.to_datetime(row['Timestamp']) - pd.Timedelta(1, 'd')).dayofweek
  dow_b = (pd.to_datetime(row['Timestamp']) - pd.Timedelta(7, 'd')).dayofweek
  if (dow_a >= 5):
    date_a = 3
  o = temp.loc[(pd.to_datetime(temp['Timestamp']) == (pd.to_datetime(row['Timestamp']) - pd.Timedelta(date_a, 'd')))]['Lowest Price'].mean()
  c = temp2.loc[(pd.to_datetime(temp2['Timestamp']) == (pd.to_datetime(row['Timestamp']) - pd.Timedelta(date_b, 'd')))]['Lowest Price'].mean()
  if isnan((c - o)/o):
    return 0
  else:
    return round((c - o)/o, 2)

def stock_change_3d (row):
  date_a = 1
  date_b = 3
  dow_a = (pd.to_datetime(row['Timestamp']) - pd.Timedelta(1, 'd')).dayofweek
  dow_b = (pd.to_datetime(row['Timestamp']) - pd.Timedelta(3, 'd')).dayofweek

  if (dow_a < 5) & (dow_b < 5): # Neither are weekends
    date_a = 1
    date_b = 3
  elif (dow_a >= 5): # Day before is Sunday, so 3-days before cannot be weekend
    date_a = 3
    date_b = 5
  else: # 3-days before is weekend
    date_a = 1
    date_b = 3 + dow_b - 4
  o = temp.loc[(pd.to_datetime(temp['Timestamp']) == (pd.to_datetime(row['Timestamp']) - pd.Timedelta(date_a, 'd')))]['Lowest Price'].mean()
  c = temp2.loc[(pd.to_datetime(temp2['Timestamp']) == (pd.to_datetime(row['Timestamp']) - pd.Timedelta(date_b, 'd')))]['Lowest Price'].mean()
  if isnan((c - o)/o):
    return 0
  else:
    return round((c - o)/o, 2)

def stock_change_1d (row):
  # This is always 0
  return 0

df['SPY 5D Stock Change'] = df.apply(stock_change_5d, axis = 1)
df['SPY 3D Stock Change'] = df.apply(stock_change_3d, axis = 1)
df['SPY 1D Stock Change'] = df.apply(stock_change_1d, axis = 1)

temp = df_stock_open_5d.loc[df_stock_open_5d['Stock Name'] == 'QQQ']
temp2 = df_stock_close_5d.loc[df_stock_close_5d['Stock Name'] == 'QQQ']
df['QQQ 5D Stock Change'] = df.apply(stock_change_5d, axis = 1)
df['QQQ 3D Stock Change'] = df.apply(stock_change_3d, axis = 1)
df['QQQ 1D Stock Change'] = df.apply(stock_change_1d, axis = 1)

temp = df_stock_open_5d.loc[df_stock_open_5d['Stock Name'] == 'VXX']
temp2 = df_stock_close_5d.loc[df_stock_close_5d['Stock Name'] == 'VXX']
df['VXX 5D Stock Change'] = df.apply(stock_change_5d, axis = 1)
df['VXX 3D Stock Change'] = df.apply(stock_change_3d, axis = 1)
df['VXX 1D Stock Change'] = df.apply(stock_change_1d, axis = 1)

temp = df_stock_open_5d.loc[df_stock_open_5d['Stock Name'] == 'DIA']
temp2 = df_stock_close_5d.loc[df_stock_close_5d['Stock Name'] == 'DIA']
df['DIA 5D Stock Change'] = df.apply(stock_change_5d, axis = 1)
df['DIA 3D Stock Change'] = df.apply(stock_change_3d, axis = 1)
df['DIA 1D Stock Change'] = df.apply(stock_change_1d, axis = 1)

### Strike
# Uncomment when above is done
from math import ceil
df['SPY Strike'] = df['SPY Closing Price'].apply(ceil)
df['QQQ Strike'] = df['QQQ Closing Price'].apply(ceil)
df['VXX Strike'] = df['VXX Closing Price'].apply(ceil)
df['DIA Strike'] = df['DIA Closing Price'].apply(ceil)

df.head(15)

,Timestamp,SPY Opening Price,QQQ Opening Price,VXX Opening Price,DIA Opening Price,SPY Closing Price,QQQ Closing Price,VXX Closing Price,DIA Closing Price,SPY 5D Avg Volume,...,VXX 5D Stock Change,VXX 3D Stock Change,VXX 1D Stock Change,DIA 5D Stock Change,DIA 3D Stock Change,DIA 1D Stock Change,SPY Strike,QQQ Strike,VXX Strike,DIA Strike
0,2022-01-03,477.26,399.85,72.56,364.79,477.3800,401.6800,71.76,365.24,5.562423e+07,...,0.06,0.01,0,0.00,0.00,0,478,402,72,366
1,2022-01-04,479.11,402.13,71.04,367.08,477.1700,394.8700,71.48,368.26,5.879615e+07,...,0.07,0.02,0,-0.00,-0.00,0,478,395,72,369
2,2022-01-05,477.09,394.92,71.88,367.94,471.1100,386.9400,74.90,365.38,6.361537e+07,...,0.06,0.04,0,-0.01,-0.01,0,472,387,75,366
3,2022-01-06,468.35,382.37,77.08,364.96,468.9900,385.0000,76.48,362.84,7.369606e+07,...,0.02,-0.00,0,-0.01,-0.01,0,469,385,77,363
4,2022-01-07,466.42,381.16,77.60,361.70,467.1100,380.5707,75.48,363.34,7.992996e+07,...,-0.04,-0.07,0,-0.00,0.01,0,468,381,76,364
5,2022-01-10,462.98,375.19,77.88,361.26,462.9400,376.9100,76.92,359.59,8.396879e+07,...,-0.08,-0.03,0,0.01,0.01,0,463,377,77,360
6,2022-01-11,465.86,380.97,74.08,360.74,468.6500,385.0800,71.36,362.08,9.330754e+07,...,-0.08,-0.03,0,0.02,0.01,0,469,386,72,363
7,2022-01-12,471.42,387.36,70.24,364.10,470.2900,386.8700,70.84,362.31,9.393132e+07,...,0.01,0.02,0,0.01,0.01,0,471,387,71,363
8,2022-01-13,471.65,387.85,70.28,364.02,466.8400,380.1000,72.98,362.60,8.655282e+07,...,0.09,0.10,0,-0.00,-0.01,0,467,381,73,363
9,2022-01-14,460.88,373.88,76.88,358.56,462.0650,377.6600,75.56,357.44,8.748064e+07,...,0.07,0.02,0,-0.00,-0.01,0,463,378,76,358


In [ ]:

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 0 to 495
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Timestamp            496 non-null    object 
 1   SPY Opening Price    496 non-null    float64
 2   QQQ Opening Price    496 non-null    float64
 3   VXX Opening Price    496 non-null    float64
 4   DIA Opening Price    496 non-null    float64
 5   SPY Closing Price    496 non-null    float64
 6   QQQ Closing Price    496 non-null    float64
 7   VXX Closing Price    496 non-null    float64
 8   DIA Closing Price    496 non-null    float64
 9   SPY 5D Avg Volume    496 non-null    float64
 10  SPY 3D Avg Volume    496 non-null    float64
 11  SPY 1D Avg Volume    496 non-null    float64
 12  QQQ 5D Avg Volume    496 non-null    float64
 13  QQQ 3D Avg Volume    496 non-null    float64
 14  QQQ 1D Avg Volume    496 non-null    float64
 15  VXX 5D Avg Volume    496 non-null    flo

In [ ]:
# CSV save - This is for my own account
from google.colab import drive
# drive.mount('/content/drive')
# df.to_csv('/content/drive/My Drive/df.csv', index=False)
# df_verify = pd.read_csv('/content/drive/My Drive/df.csv')
# df_verify.head()
# drive.flush_and_unmount()

In [ ]:
## Step 6 - Grab Options DF (Cannot be achieved as of now, because we need strike price generated from Step 5)

# Option code = O:{stocksTicker}{optionDate}C00{strike}000
# If strike is less than 3-digit, add 0 in front of {strike} per digit missing, and vice versa.
# optionDate is 6-digit number of YYMMDD without hyphens. (2024-03-18 => 240318)

def cc_open_spy(row):
  string = str(row['Timestamp'])
  odate = string[2:4] + "" + string[5:7] + "" + string[8:] # 6-digit code for date
  strike = str(row['SPY Strike'])
  if len(strike) == 2:
    strike = '0' + strike
  ocode = f'O:SPY{odate}C00{strike}000'
  query_url = f'{base_url}/v2/aggs/ticker/{ocode}/range/5/minute/{string}/{string}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}'
  response = requests.get(query_url)
  json_data = response.json()
  try:
    if json_data['resultsCount'] > 0:
        # Convert JSON to Pandas Dataframe
        tdf = pd.json_normalize(json_data['results'])
        tdf['t'] = pd.to_datetime(tdf['t'], unit='ms')
        oprice = tdf.loc[(tdf['t'].dt.hour == 13) & (tdf['t'].dt.minute == 35)]['l'].mean()
        return oprice
    else:
      return 0
  except:
    return -1

def cc_close_spy(row):
  string = str(row['Timestamp'])
  odate = string[2:4] + "" + string[5:7] + "" + string[8:] # 6-digit code for date
  strike = str(row['SPY Strike'])
  if len(strike) == 2:
    strike = '0' + strike
  ocode = f'O:SPY{odate}C00{strike}000'
  query_url = f'{base_url}/v2/aggs/ticker/{ocode}/range/5/minute/{string}/{string}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}'
  response = requests.get(query_url)
  json_data = response.json()
  try:
    if json_data['resultsCount'] > 0:
        # Convert JSON to Pandas Dataframe
        tdf = pd.json_normalize(json_data['results'])
        tdf['t'] = pd.to_datetime(tdf['t'], unit='ms')
        cprice = tdf.loc[(tdf['t'].dt.hour == 19) & (tdf['t'].dt.minute == 45)]['h'].mean()
        return cprice
    else:
      return 0
  except:
    return -1

def cc_open_qqq(row):
  string = str(row['Timestamp'])
  odate = string[2:4] + "" + string[5:7] + "" + string[8:] # 6-digit code for date
  strike = str(row['QQQ Strike'])
  if len(strike) == 2:
    strike = '0' + strike
  ocode = f'O:QQQ{odate}C00{strike}000'
  query_url = f'{base_url}/v2/aggs/ticker/{ocode}/range/5/minute/{string}/{string}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}'
  response = requests.get(query_url)
  json_data = response.json()
  try:
    if json_data['resultsCount'] > 0:
        # Convert JSON to Pandas Dataframe
        tdf = pd.json_normalize(json_data['results'])
        tdf['t'] = pd.to_datetime(tdf['t'], unit='ms')
        oprice = tdf.loc[(tdf['t'].dt.hour == 13) & (tdf['t'].dt.minute == 35)]['l'].mean()
        return oprice
    else:
      return 0
  except:
    return -1

def cc_close_qqq(row):
  string = str(row['Timestamp'])
  odate = string[2:4] + "" + string[5:7] + "" + string[8:] # 6-digit code for date
  strike = str(row['QQQ Strike'])
  if len(strike) == 2:
    strike = '0' + strike
  ocode = f'O:QQQ{odate}C00{strike}000'
  query_url = f'{base_url}/v2/aggs/ticker/{ocode}/range/5/minute/{string}/{string}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}'
  response = requests.get(query_url)
  json_data = response.json()
  try:
    if json_data['resultsCount'] > 0:
        # Convert JSON to Pandas Dataframe
        tdf = pd.json_normalize(json_data['results'])
        tdf['t'] = pd.to_datetime(tdf['t'], unit='ms')
        cprice = tdf.loc[(tdf['t'].dt.hour == 19) & (tdf['t'].dt.minute == 45)]['h'].mean()
        return cprice
    else:
      return 0
  except:
    return -1

def cc_open_vxx(row):
  string = str(row['Timestamp'])
  odate = string[2:4] + "" + string[5:7] + "" + string[8:] # 6-digit code for date
  strike = str(row['VXX Strike'])
  if len(strike) == 2:
    strike = '0' + strike
  ocode = f'O:VXX{odate}C00{strike}000'
  query_url = f'{base_url}/v2/aggs/ticker/{ocode}/range/5/minute/{string}/{string}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}'
  response = requests.get(query_url)
  json_data = response.json()
  try:
    if json_data['resultsCount'] > 0:
        # Convert JSON to Pandas Dataframe
        tdf = pd.json_normalize(json_data['results'])
        tdf['t'] = pd.to_datetime(tdf['t'], unit='ms')
        oprice = tdf.loc[(tdf['t'].dt.hour == 13) & (tdf['t'].dt.minute == 35)]['l'].mean()
        return oprice
    else:
      return 0
  except:
    return -1

def cc_close_vxx(row):
  string = str(row['Timestamp'])
  odate = string[2:4] + "" + string[5:7] + "" + string[8:] # 6-digit code for date
  strike = str(row['VXX Strike'])
  if len(strike) == 2:
    strike = '0' + strike
  ocode = f'O:VXX{odate}C00{strike}000'
  query_url = f'{base_url}/v2/aggs/ticker/{ocode}/range/5/minute/{string}/{string}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}'
  response = requests.get(query_url)
  json_data = response.json()
  try:
    if json_data['resultsCount'] > 0:
        # Convert JSON to Pandas Dataframe
        tdf = pd.json_normalize(json_data['results'])
        tdf['t'] = pd.to_datetime(tdf['t'], unit='ms')
        cprice = tdf.loc[(tdf['t'].dt.hour == 19) & (tdf['t'].dt.minute == 45)]['h'].mean()
        return cprice
    else:
      return 0
  except:
    return -1

def cc_open_dia(row):
  string = str(row['Timestamp'])
  odate = string[2:4] + "" + string[5:7] + "" + string[8:] # 6-digit code for date
  strike = str(row['DIA Strike'])
  if len(strike) == 2:
    strike = '0' + strike
  ocode = f'O:DIA{odate}C00{strike}000'
  query_url = f'{base_url}/v2/aggs/ticker/{ocode}/range/5/minute/{string}/{string}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}'
  response = requests.get(query_url)
  json_data = response.json()
  try:
    if json_data['resultsCount'] > 0:
        # Convert JSON to Pandas Dataframe
        tdf = pd.json_normalize(json_data['results'])
        tdf['t'] = pd.to_datetime(tdf['t'], unit='ms')
        oprice = tdf.loc[(tdf['t'].dt.hour == 13) & (tdf['t'].dt.minute == 35)]['l'].mean()
        return oprice
    else:
      return 0
  except:
    return -1

def cc_close_dia(row):
  string = str(row['Timestamp'])
  odate = string[2:4] + "" + string[5:7] + "" + string[8:] # 6-digit code for date
  strike = str(row['DIA Strike'])
  if len(strike) == 2:
    strike = '0' + strike
  ocode = f'O:DIA{odate}C00{strike}000'
  query_url = f'{base_url}/v2/aggs/ticker/{ocode}/range/5/minute/{string}/{string}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}'
  response = requests.get(query_url)
  json_data = response.json()
  try:
    if json_data['resultsCount'] > 0:
        # Convert JSON to Pandas Dataframe
        tdf = pd.json_normalize(json_data['results'])
        tdf['t'] = pd.to_datetime(tdf['t'], unit='ms')
        cprice = tdf.loc[(tdf['t'].dt.hour == 19) & (tdf['t'].dt.minute == 45)]['h'].mean()
        return cprice
    else:
      return 0
  except:
    return -1


testtest = df.copy()
testtest['SPY Open CC'] = testtest.apply(cc_open_spy, axis = 1)
testtest['SPY Close CC'] = testtest.apply(cc_close_spy, axis = 1)
testtest['QQQ Open CC'] = testtest.apply(cc_open_qqq, axis = 1)
testtest['QQQ Close CC'] = testtest.apply(cc_close_qqq, axis = 1)
testtest['VXX Open CC'] = testtest.apply(cc_open_vxx, axis = 1)
testtest['VXX Close CC'] = testtest.apply(cc_close_vxx, axis = 1)
testtest['DIA Open CC'] = testtest.apply(cc_open_dia, axis = 1)
testtest['DIA Close CC'] = testtest.apply(cc_close_dia, axis = 1)
testtest.tail(30)

,Timestamp,SPY Opening Price,QQQ Opening Price,VXX Opening Price,DIA Opening Price,SPY Closing Price,QQQ Closing Price,VXX Closing Price,DIA Closing Price,SPY 5D Avg Volume,...,VXX Strike,DIA Strike,SPY Open CC,SPY Close CC,QQQ Open CC,QQQ Close CC,VXX Open CC,VXX Close CC,DIA Open CC,DIA Close CC
466,2024-01-18,474.47,410.790,15.4000,372.320,474.9400,412.1100,15.3850,373.700,72398793.50,...,16,374,NaN,0.81,NaN,0.43,0.00,0.00,0.0,0.00
467,2024-01-19,479.02,416.300,15.0700,376.730,481.6200,420.2300,15.0400,378.480,75885352.25,...,16,379,NaN,0.63,NaN,0.49,NaN,NaN,NaN,0.34
468,2024-01-25,486.79,427.420,14.4000,378.600,485.8000,424.5785,14.5600,378.550,81971519.60,...,15,379,NaN,0.53,NaN,0.58,0.00,0.00,0.0,0.00
469,2024-01-26,486.95,424.000,14.6400,379.490,487.7050,424.3304,14.3900,380.910,78101267.80,...,15,381,NaN,0.42,NaN,0.32,NaN,NaN,NaN,0.29
470,2024-01-31,488.38,420.890,14.3200,384.930,487.7400,421.5100,14.5550,384.800,70048347.40,...,15,385,NaN,1.60,NaN,1.59,0.00,0.00,0.0,0.00
471,2024-02-01,484.79,419.370,14.8300,381.690,487.8200,421.1600,14.7800,384.190,78905551.80,...,15,385,NaN,1.18,NaN,1.10,0.00,0.00,0.0,0.00
472,2024-02-02,489.83,423.900,14.8000,384.010,495.3700,429.7900,15.0400,387.380,82782881.40,...,16,388,NaN,0.42,NaN,0.68,NaN,NaN,NaN,0.15
473,2024-02-07,495.33,429.300,13.9100,385.770,497.9700,431.9200,13.9500,386.960,89745318.80,...,14,387,NaN,0.63,NaN,0.70,0.00,0.00,0.0,0.00
474,2024-02-09,500.16,434.520,13.8200,388.580,501.3800,437.6100,14.0241,387.100,70709939.60,...,15,388,NaN,0.27,NaN,0.36,NaN,0.01,NaN,0.07
475,2024-02-13,495.30,428.010,14.6400,385.101,491.6900,426.7400,15.4400,380.750,59745212.40,...,16,381,NaN,0.91,NaN,0.89,0.00,0.00,0.0,0.00


In [ ]:
testtest['SPY Close CC'].info()

<class 'pandas.core.series.Series'>
Int64Index: 496 entries, 0 to 495
Series name: SPY Close CC
Non-Null Count  Dtype  
--------------  -----  
496 non-null    float64
dtypes: float64(1)
memory usage: 7.8 KB


In [ ]:
# CSV save - This is for my own account
from google.colab import drive
#drive.mount('/content/drive')
#testtest.to_csv('/content/drive/My Drive/df_final.csv', index=False)
#df_verify = pd.read_csv('/content/drive/My Drive/df_final.csv')
#df_verify.head()
drive.flush_and_unmount()